# Fact Retreival Bot using IDFT
### Steps
- Loading and preprocessing Questions and Answers from dataset
- Setting Stopwords
- Intitialising and training TF_IDF vectors
- Testing

## Imports

In [4]:
import pandas as pd                   # To load and process dataset
import numpy as np                    # For matrix operations
from nltk.corpus import stopwords     # Using NLTK to load stopwords
from nltk import word_tokenize        # Using NLTK to token sentences

from beakerx import *
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.width',1000)

## Loading and preprocessing Questions and Answers from dataset
- `hdfc.pkl` : Collection of 1341 QnA about HDFC. (Scraped from HDFC's FAQ site)
- Dropping stopwords
- Stripping Questions of extra spaces

In [5]:
df = pd.read_excel('hdfc.xlsx')
df = df.drop_duplicates('Question')
df = df.reset_index()

ImportError: Install xlrd >= 0.9.0 for Excel support

In [ ]:
limit = 1000
reduced = df[['Question','Answer']][:limit]

qlabels = reduced['Question'].to_dict()
alabels = reduced['Answer'].to_dict()

print reduced.head()

## Setting stopwords
- Import set of common stopwords from nltk
- Adding domain-related stopword
- Removing question words (To distinguish between intents of questions)

In [ ]:
plus = {'hdfc'}
minus = {'what','how','where','when','why'}
stop = set(stopwords.words('english'))

stop.update(plus)
stop.difference_update(minus)

## Intitialising and training TF-IDF vectors
- Setting stopwords to `stop`
- `tf_vect` : `TfidfVectorizer` object. Can be used to convert strings to tf-idf vectors
- `all_qs_vectors` : Matrix of TF-IDF vectors corresponding to questions in training set

In [ ]:
tf_vect =TfidfVectorizer(stop_words=stop,
                         lowercase=True,
                         use_idf=True)
all_qs_vectors = tf_vect.fit_transform(reduced['Question'])
print "Shape of all_qs_vectors :",all_qs_vectors.shape
print all_qs_vectors.shape[0],": Number of questions"
print all_qs_vectors.shape[1],": Vocabulary size"

In [ ]:
# Transforming context with tfidf
context = 'How can I repay my Personal Loan?'
context_vector = tf_vect.transform([context])
context_matrix = context_vector.todense()

In [ ]:
# Displaying TF_IDF results
print "WORD".ljust(10),"INDEX".ljust(6),"TFIDF_VALUE"
for w in word_tokenize(context.strip()):
    ind = tf_vect.vocabulary_.get(w.lower(),"NA")
    val = context_matrix[0,ind] if not ind == "NA" else 0
    print w.ljust(10),str(ind).ljust(6),val

## Predicting closest question
- `predict` has the following arguments
    - `n`       : int  | Number of results (from top)
    - `answers` : bool | Return answers or not
    - `ret_best`: bool | Returns index of closest match
- Steps for prediction
    - Convert query to tfidf vector
    - Get dot product of query vectors with each question to measures similarity
    - Sort array indices by descending order of array values
    - Return top n results

In [ ]:
def predict(query,n=5,answers=False,ret_indices=False):
    # Comparing context with all questions using dot product
    query_vector = tf_vect.transform([query])
    sim = np.dot(all_qs_vectors, query_vector.T)
    # Converting numpy matrix to 1D array with 146 dot products (146 questions vs context)
    arr = sim.toarray().flatten()
    matches = arr.argsort(axis=0)[::-1]
    top_n_matches = matches[:n]
    results = []
    if ret_indices:
        return top_n_matches
    for i in top_n_matches:
        res = {"Question":qlabels[i],"Ans":alabels[i]} if answers else {"Question":qlabels[i]}
        results.append(res)
    return results

In [ ]:
predict('How do I pay my personal loan ?')

In [ ]:
num_correct = 0
failed = []
for i in qlabels:
    if predict(qlabels[i],n=1,ret_indices=True)[0] == i:
        num_correct +=1
    else :
        failed.append(i)
print "Recall : ",float(num_correct)/len(qlabels) *100,"%"

for i in failed : 
    query = qlabels[i]
    print "\nQuery :",i,query
    print predict(query,n=3)


In [ ]:
# Generating tokens after converting to lowercase, removing stopwords and non-alphanumberic tokens
# Note : nltk.word_tokenize does not split PIN/Pattern'
tokens = {}
for i in qlabels:
    tokens[i] = set([x for x in wordpunct_tokenize(qlabels[i].lower()) if x.isalnum() and x not in stop])

In [ ]:
# Eliminating questions which have a jaccard_distance > 0.9 with another questions
def get_jaccard_similarity(words,words2):
    inter = words.intersection(words2)
    union = words.union(words2)
    return float(len(inter))/len(union)

def pred_jaccard(query):
    words = [x for x in wordpunct_tokenize(query.lower()) if x.isalnum() and x not in stop]
    
    

In [ ]:
TableDisplay'm3

In [ ]:
TableDisplay